# Sampling
Treating imbalance class distribution

In [88]:

# Reloading
%reload_ext autoreload
%autoreload 2

import pandas as pd
print('pandas version:{}'.format(pd.__version__))
import numpy as np
print('numpy version:{}'.format(np.__version__))
import math
from datetime import datetime
from dgpylib import dg_athena
from dgpylib import dg_s3
import os
import re
from matplotlib.ticker import MaxNLocator
from matplotlib import pyplot as plt
import seaborn as sns
from datetime import timedelta
from dateutil.relativedelta import relativedelta

import sys
sys.path.insert(0, '../Modelling')
import helper_function2 as hf

import sys
sys.path.insert(0, '../EDA')
import data_integrity_fixer as dint
import basicstatsandplotter as bsp
import data_imputer as di
import data_integrity_fixer as dif
import parameters

##for modelling
import h2o
print('h2o version:{}'.format(h2o.__version__))
from h2o.estimators import H2OXGBoostEstimator
from h2o.grid.grid_search import H2OGridSearch
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators import H2OTargetEncoderEstimator
from h2o.estimators import H2ORandomForestEstimator
from h2o.estimators import H2OGradientBoostingEstimator
from h2o.tree import H2OTree
from h2o.tree import H2ONode
from h2o.tree import H2OSplitNode
from h2o.tree import H2OLeafNode
from h2o.estimators.kmeans import H2OKMeansEstimator

#import xgboost as xgb
import sklearn
import category_encoders as ce
print('sklearn version:{}'.format(sklearn.__version__))
from sklearn import datasets
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import r2_score
from sklearn.preprocessing import PowerTransformer
import time
import parameters
from statsmodels.stats.outliers_influence import variance_inflation_factor
# import Grid Search
from h2o.grid.grid_search import H2OGridSearch

import math
from h2o.grid.grid_search import H2OGridSearch

from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

%run 'functions.ipynb'

plt.rcParams['figure.figsize'] = [10, 5]
plt.rcParams['figure.dpi'] = 100

pandas version:1.3.4
numpy version:1.20.3
h2o version:3.36.0.4
sklearn version:1.1.2


In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
conn_s3 = dg_s3.Connect('dgdatadump/DS&CA/Pricing/02 Projects/2022/Tel_Reg_2.0/Cancellation Model')

df_raw12 = conn_s3.read('TelReg_Canx_filt_12mo.csv')
df_raw9 = conn_s3.read('TelReg_Canx_filt_9mo.csv')
df_raw6 = conn_s3.read('TelReg_Canx_filt_6mo.csv')
df_raw3 = conn_s3.read('TelReg_Canx_filt_3mo.csv')

In [4]:
data_list = ['df_raw3', 'df_raw6', 'df_raw9', 'df_raw12']
data_set = [df_raw3, df_raw6, df_raw9, df_raw12]

In [5]:
# read feature importance performance pickle
feat_imp_performance_df = pd.read_pickle("feat_imp_performance_df.pkl")
# read model performance pickle
performance_df = pd.read_pickle("performance_df.pkl")

In [6]:
for i in range(len(data_set)):
    print(f'size of {data_list[i]}: {data_set[i].shape[0]} rows')

size of df_raw3: 357058 rows
size of df_raw6: 270538 rows
size of df_raw9: 167819 rows
size of df_raw12: 66895 rows


In [89]:
# initialise variables for 3/6/9/12-month cancellation cohorts

idx = 0
predictors = performance_df.loc[idx, 'feats_stored_gbm']
response = 'cancelflag'

dataset = performance_df.loc[idx, 'name']
print(f'dataset: {dataset}')
print(f'predictors: {predictors}')
print(f'number of predictors: {len(predictors)}')

dataset: df_raw3
predictors: ['ManufacturerBrandCode', 'ONS_%people_working_FT', 'ONS_avg_age', 'PlansAcceptedPast1YearCount', 'PlanLiveCount', 'ClientAccountDesc', 'PurchasePrice', 'AppAge', 'PlansActivePast5yrCount', 'ClientGroupDesc', 'ApplianceCode', 'ONS_n_families_per_population', 'Fee', 'price_diff', 'ONS_%people_working_49+', 'ONS_avg_distance_travelled_to_work(km)', 'ONS_avg_household_size', 'ONS_avg_dependent_children_per_family', 'ONS_bedrooms per rooms', 'GoodsColour']
number of predictors: 20


In [90]:
df = data_set[idx].copy()

## Use SMOTE

In [91]:
cat_predictors = df[predictors].select_dtypes(include=['object']).columns.tolist()
cat_predictors

['ManufacturerBrandCode',
 'ClientAccountDesc',
 'ClientGroupDesc',
 'ApplianceCode',
 'GoodsColour']

In [92]:
# apply leave one out encoder to non-ordinal feats
for feature in ['ManufacturerBrandCode','ClientAccountDesc','ApplianceCode','ClientGroupDesc','GoodsColour']:
    l1o_encoder, df[feature] = get_leave_one_out(df[feature], df['cancelflag'])
    
# # apply get_ordinal to ordinal feats
# for feature in []:
#     ord_encoder, df_[feature] = get_ordinal(df_[feature])

In [93]:
X = df[predictors]
y = df[response]

In [94]:
data_set[idx][response].value_counts()

0    312570
1     44488
Name: cancelflag, dtype: int64

In [95]:
# instantiating over and under sampler
over = RandomOverSampler(sampling_strategy=0.5)
under = RandomUnderSampler(sampling_strategy=0.5)

# first performing oversampling to minority class
X_over, y_over = over.fit_resample(X, y)
print(f"Oversampled: {Counter(y_over)}")

Oversampled: Counter({0: 312570, 1: 156285})


In [96]:
# now to combine under sampling 
X_combined_sampling, y_combined_sampling = under.fit_resample(X_over, y_over)
print(f"Combined Random Sampling: {Counter(y_combined_sampling)}")

Combined Random Sampling: Counter({0: 312570, 1: 156285})


In [97]:
df = pd.concat([X_combined_sampling, y_combined_sampling], axis=1)

In [ ]:
# sm = SMOTE(random_state=42)
# X_res, y_res = sm.fit_resample(X, y)
# print('Resampled dataset shape %s' % Counter(y_res))

In [85]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,23 mins 44 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.0.4
H2O_cluster_version_age:,5 months and 3 days !!!
H2O_cluster_name:,H2O_from_python_ubuntu_89xuq0
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,29.88 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


In [98]:
# convert columns to factors
for i in df.columns[0:]:
    df[i] = df[i].astype('category')

In [99]:

# re train test split
x_train, x_test, y_train, y_test = train_test_split(df[predictors], df[response], test_size = 0.2, random_state = 1)
train = pd.concat([x_train, y_train], axis=1)
test = pd.concat([x_test, y_test], axis=1)

# preserve training & test data's index to be used as a key to merge prediction probability with main dataset 
xtrain_index = x_train.index
xtest_index = x_test.index

# convert to h2o dataframe format
hf_train = h2o.H2OFrame(train)
hf_test = h2o.H2OFrame(test)

# format target to fit the model
hf_train['cancelflag']=hf_train['cancelflag'].asfactor()
hf_test['cancelflag']=hf_test['cancelflag'].asfactor()

# initialise the estimator 
df_gbm = H2OGradientBoostingEstimator(keep_cross_validation_predictions=True, nfolds = 5, seed = 1)

start = time.time()
# train the model
df_gbm.train(x = predictors, y = response, training_frame = hf_train)
end = time.time()
print(end - start)

# classification report
y_true = y_test
y_pred = df_gbm.predict(hf_test)
y_pred = y_pred.as_data_frame()

print(classification_report(y_true, y_pred['predict'], labels=[0,1]))

# performance metrics
print_model_perf_stats(df_gbm, hf_train, hf_test)

# AUC of cross-validated holdout predictions
print(f'AUC of cross-validated holdout predictions: {df_gbm.auc(xval=True)}')

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
29.510124444961548
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
              precision    recall  f1-score   support

           0       0.79      0.28      0.41     62675
           1       0.37      0.85      0.52     31096

    accuracy                           0.47     93771
   macro avg       0.58      0.57      0.46     93771
weighted avg       0.65      0.47      0.45     93771

MCC: 0.17074 / 0.16770
F1: 0.52001 / 0.51571
AUC: 0.63185 / 0.62689
AUC PR: 0.48111 / 0.47262
Accuracy: 0.68287 / 0.68209
Logloss: 0.60864 / 0.60881
KS: 0.17972
AUC of cross-validated holdout predictions: 0.6241710863153502


## Using H2O balance_classes

In [40]:
# convert columns to factors
for i in data_set[idx].columns[0:]:
    df[i] = data_set[idx][i].astype('category')

# re train test split
x_train, x_test, y_train, y_test = train_test_split(df[predictors], df[response], test_size = 0.2, random_state = 1)
train = pd.concat([x_train, y_train], axis=1)
test = pd.concat([x_test, y_test], axis=1)

# preserve training & test data's index to be used as a key to merge prediction probability with main dataset 
xtrain_index = x_train.index
xtest_index = x_test.index

# convert to h2o dataframe format
hf_train = h2o.H2OFrame(train)
hf_test = h2o.H2OFrame(test)

# format target to fit the model
hf_train['cancelflag']=hf_train['cancelflag'].asfactor()
hf_test['cancelflag']=hf_test['cancelflag'].asfactor()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


### with class_sampling_factors

In [ ]:
# try using the class_sampling_factors parameter:
# since all but Class 2 have similar frequency counts, let's undersample Class 2
# and not change the sampling rate of the other classes
# note: class_sampling_factors must be a list of floats
sample_factors = [1., 0.23]
balance_gbm = H2OGradientBoostingEstimator(balance_classes = True, class_sampling_factors = sample_factors, seed = 1234)

start = time.time()
balance_gbm.train(x = predictors, y = response, training_frame = hf_train, validation_frame = hf_test)
end = time.time()
print(end - start)

In [ ]:
# select the values for `class_sampling_factors` to grid over

hyper_params = {'class_sampling_factors': [[1., 1.867208334], [1., 0.23], [0.535558878, 1.]]}

In [ ]:
# this example uses cartesian grid search because the search space is small
# and we want to see the performance of all models. For a larger search space use
# random grid search instead: {'strategy': "RandomDiscrete"}
# initialize the GBM estimator
balance_gbm_2 = H2OGradientBoostingEstimator(balance_classes = True, seed = 1234)

In [ ]:
# build grid search with previously made GBM and hyperparameters
grid = H2OGridSearch(model = balance_gbm_2, hyper_params = hyper_params,
                     search_criteria = {'strategy': "Cartesian"})

# train using the grid
start = time.time()
grid.train(x = predictors, y = response, training_frame = hf_train, validation_frame = hf_test)
end = time.time()
print(end - start)

In [ ]:
# sort the grid models by auc
sorted_grid = grid.get_grid(sort_by='auc', decreasing=False)
print(sorted_grid)

### without class_sampling_factors

In [ ]:
# try using the balance_classes parameter (set to True):
balance_gbm = H2OGradientBoostingEstimator(balance_classes = True, seed = 1234)

# train GBM model
start = time.time()
balance_gbm.train(x = predictors, y = response, training_frame = hf_train, validation_frame = hf_test)
end = time.time()
print(end - start)


In [ ]:
# select the values for `balance_classes` to grid over
hyper_params = {'balance_classes': [True, False]}

In [ ]:
# this example uses cartesian grid search because the search space is small
# and we want to see the performance of all models. For a larger search space use
# random grid search instead: {'strategy': "RandomDiscrete"}
# initialize the GBM estimator
balance_gbm_2 = H2OGradientBoostingEstimator(seed = 1234)

In [ ]:
# build grid search with previously made GBM and hyperparameters
grid = H2OGridSearch(model = balance_gbm_2, hyper_params = hyper_params,
                     search_criteria = {'strategy': "Cartesian"})

start = time.time()
# train using the grid
grid.train(x = predictors, y = response, training_frame = hf_train, validation_frame = hf_test)
end = time.time()
print(end - start)

In [ ]:
# sort the grid models by auc
sorted_grid = grid.get_grid(sort_by='auc', decreasing=True)
print(sorted_grid)

In [ ]:
# end of sampling

In [ ]:
# Build a GBM model on the whole dataset (instead of the 60%) and using internal 5-fold cross-validation (re-using all other parameters including the seed)
gbm = h2o.get_model(sorted_grid.sorted_metric_table()['model_ids'][0])

#get the parameters from the Random grid search model and modify them slightly
params = gbm.params
new_params = {"nfolds":5, "model_id":None, "training_frame":None, "validation_frame":None, 
              "response_column":None, "ignored_columns":None}
for key in new_params.keys():
    params[key]['actual'] = new_params[key] 
gbm_best = H2OGradientBoostingEstimator()
for key in params.keys():
    if key in dir(gbm_best) and getattr(gbm_best,key) != params[key]['actual']:
        setattr(gbm_best,key,params[key]['actual'])

In [ ]:
hf_df = h2o.H2OFrame(df)

In [ ]:
# format target to fit the model
hf_df['cancelflag']=hf_df['cancelflag'].asfactor()

# train GBM model on the whole dataset
start = time.time()
gbm_best.train(x=predictors, y=response, training_frame=hf_df)
end = time.time()
print(end - start)

print(gbm_best.cross_validation_metrics_summary())

In [ ]:
# classification report
y_true = y_test
y_pred = gbm_best.predict(hf_test)
y_pred = y_pred.as_data_frame()

print(classification_report(y_true, y_pred['predict'], labels=[0,1]))

In [ ]:
y_test.astype(int).values.mean()

In [ ]:
y_train.astype(int).values.mean()

In [ ]:
model_perf_stats(gbm_best, hf_train, hf_test)

In [ ]:
path = 'GBM_cancellation_model_12mo'
# load the model
tuned_gbm_best =  h2o.import_mojo(path)

In [ ]:
sorted_grid[0]